# S3VT Landsat and Sentinel 2 validation of hotspots - working

## Description
This notebook demonstrates how to:
 
From a candidate latitude longitude and solar_day:
* determine if intersecting Landsat or Sentinel 2 ARD exists
* apply the platform specific tests to determine if hotspots were detected in the vicinity 5km of hotspot
* return number of pixel identified as hotspots
* save a boolean file labelled with solar date of acquisition
* as a secondary test perform a Normalized Burnt Ratio and return as a binary with solar date of acquisition
    * find canidate dates within a time range of source hotspot
        * find closest before date within tolerance (dNBR A)
        * find closest after date within tolerance (dNBR B)
        * candidate closest to source hotspot will be used for hotspot matching i.e. high resolution hotspot

Assumptions:
* reflectance values are scaled by 10000 i.e. 100% reflectance = 10000
 

### Load packages

In [ ]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import sys
import xarray as xr

sys.path.append("Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_bandindices import calculate_indices


### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='validating_hotspots')

In [ ]:
# Buffer candidate hotspot with 5 kilometre radius (or .05 degrees will do)
def buffer_hotspot(lon, lat):
    ul_lon = lon - 0.05
    lr_lon = lon + 0.05
    ul_lat = lat + 0.05
    lr_lat = lat - 0.05
    return ((ul_lon, lr_lon), (ul_lat, lr_lat))

### Be ignorant of the sensor

In [ ]:
# configure sensor bands - #TODO implement sensor ignorance code here
sensor_ignorance = {'msi':{'0.433-0.453': 'nbar_coastal_aerosol',
                           '0.450-0.515': 'nbar_blue',
                           '0.525-0.600': 'nbar_green',
                           '0.630-0.680': 'nbar_red',
                           '0.845-0.885': 'nbar_nir_1',
                           '1.560-1.660': 'nbar_swir_2',
                           '2.100-2.300': 'nbar_swir_3'},
                   'oli': {'0.433-0.453': 'nbart_band01',
                           '0.450-0.515': 'nbart_band02',
                           '0.525-0.600': 'nbart_band03',
                           '0.630-0.680': 'nbart_band04',
                           '0.845-0.885': 'nbart_band05',
                           '1.560-1.660': 'nbart_band06',
                           '2.100-2.300': 'nbart_band07'}}

In [ ]:
# arguments
hotspot_lon = 147.92 #149.93 #150.8
hotspot_lat = -37.48 #-34.08 #-32.65
hotspot_solar_day = "2019-12-16","2019-12-22" #"2019-12-14","2019-12-20" #"2019-12-31"#, "2020-06-10" #, '2020-01-20'
sensors_products = {'oli': ['ga_ls8c_ard_3'], 'msi': ['ga_s2a_ard_nbar_granule', 'ga_s2b_ard_nbar_granule']}
xtuple, ytuple = buffer_hotspot(hotspot_lon, hotspot_lat)
sensor = 'msi'
measurements = []
for measurement in sensor_ignorance[sensor]:
    measurements.append(sensor_ignorance[sensor][measurement])

In [ ]:
def get_measurement_list(ds, product):
    measurement_list = []
    for i in dc.list_products().name:
        for j in dc.list_measurements().query('product == @i').name:
            if i == product:
                measurement_list.append([i, '--',j])
    return(measurement_list)

In [ ]:
# potentially unambiguous active fire pixels
def get_candidates(ds):
    test1 = (((ds[sensor_ignorance[sensor]['2.100-2.300']] / ds[sensor_ignorance[sensor]['0.845-0.885']]) > 2.5) *
             ((ds[sensor_ignorance[sensor]['2.100-2.300']]  - ds[sensor_ignorance[sensor]['0.845-0.885']]) > 3000) *
             (ds[sensor_ignorance[sensor]['2.100-2.300']] > 5000))
    # Unambiguous fire pixels
    test2 = (((ds[sensor_ignorance[sensor]['1.560-1.660']] > 8000) *
              (ds[sensor_ignorance[sensor]['0.433-0.453']] < 2000)) *
             ((ds[sensor_ignorance[sensor]['0.845-0.885']] > 4000) +
              (ds[sensor_ignorance[sensor]['2.100-2.300']] < 1000)).clip(min=0, max=1))
    # other candidate fire pixels
    test3 = (((ds[sensor_ignorance[sensor]['2.100-2.300']]/ds[sensor_ignorance[sensor]['0.845-0.885']]) > 1.8)*
             (ds[sensor_ignorance[sensor]['2.100-2.300']]-ds[sensor_ignorance[sensor]['0.845-0.885']]  > 1700))
    unambiguous = (test1 + test2 + test3).clip(min=0, max=1)
    return(unambiguous)

In [ ]:
def get_context_kernel_array(y, x, array):
    
    T, Y, X = array.shape

    ymin = y - 60
    ymax = y + 60
    xmin = x - 60
    xmax = x + 60
    
    if ymin < 0:
        ymin = 0
    
    if xmin < 0:
        xmin = 0

    if ymax > Y:
        ymax = Y
        
    if xmax > X:
        xmax = X

    try:
        outarray = array[0][:, ymin:ymax][xmin:xmax]
    except:
        outarray = np.nans((61,61), dtype=np.float64)
    
    return(outarray, (ymin, ymax, xmin, xmax))

In [ ]:
def run_test6(ds):
    #6. ratio b7 b6 > 1.6
    return((ds[sensor_ignorance[sensor]['2.100-2.300']]/ds[sensor_ignorance[sensor]['1.560-1.660']]) > 1.6 )         
    

In [ ]:
# Oceans test
#7. {b4 > b5 AND b5 > b6 AND b6 > b7 AND b1 - b7 < 0.2}
def run_test7(ds):
    test7 = ((ds[sensor_ignorance[sensor]['0.630-0.680']]>ds[sensor_ignorance[sensor]['0.845-0.885']])*
             (ds[sensor_ignorance[sensor]['0.845-0.885']]>ds[sensor_ignorance[sensor]['1.560-1.660']])*
             (ds[sensor_ignorance[sensor]['1.560-1.660']]>ds[sensor_ignorance[sensor]['2.100-2.300']])*
             ((ds[sensor_ignorance[sensor]['0.433-0.453']]-ds[sensor_ignorance[sensor]['2.100-2.300']]) < 2000))

    return(test7.clip(min=0, max=1))#.plot()


In [ ]:
# Water bodies test - comment - seems like  bad test / smoke complications?
#AND
#8. {(b3 > b2)
def run_test8(ds):    
    test8 = (ds[sensor_ignorance[sensor]['0.525-0.600']]>ds[sensor_ignorance[sensor]['0.450-0.515']])

    return(test8.clip(min=0, max=1))

In [ ]:
#OR
#9. (b1 > b2 AND b2 > b3 AND b3 < b4)}.

def run_test9(ds):
    test9 = ((ds[sensor_ignorance[sensor]['0.433-0.453']]>ds[sensor_ignorance[sensor]['0.450-0.515']]) *
            (ds[sensor_ignorance[sensor]['0.450-0.515']]>ds[sensor_ignorance[sensor]['0.525-0.600']])*
            (ds[sensor_ignorance[sensor]['0.525-0.600']]<ds[sensor_ignorance[sensor]['0.630-0.680']]))
  
    return(test9.clip(min=0, max=1))


In [ ]:
def get_watermasks(ds):
    watermask=(run_test7(ds)+run_test8(ds)+run_test9(ds)).clip(min=0, max=1)
    return(watermask)

In [ ]:
def get_hotspots(ds):
    # Find the candidates and perform context check
    # TODO create mask that is ref7 > 0, non water and not other candidates
    # mask
    # b7=<0
    # water
    # othercandidates
    #candidates = (test1 + test2 + test3).clip(min=0, max=1)
    candidates = get_candidates(ds)
    watermasks = get_watermasks(ds)
    indices = np.where(candidates.data == 1)
    swircandidates = (ds[sensor_ignorance[sensor]['2.100-2.300']].where(candidates.data == 0)).where(watermasks.data == 0)
    nircandidates = (ds[sensor_ignorance[sensor]['0.845-0.885']].where(candidates.data == 0)).where(watermasks.data == 0)

    test4 = (candidates*0)
    test5 = (candidates*0)

    index = 0

    while index < len(indices[1]):
        y = indices[1][index]
        x = indices[2][index]

        #4. ratio between b7 b5 > ratio b7 b5 + max[3x std ratio b7 and b5, 0.8 ]
        #AND
        #5. b7 > b7 + max[3x std b7, 0.08]
        #AND

        #swirkernel = get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['2.100-2.300']].data)[0]
        #nirkernel = get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['0.845-0.885']].data)[0]
        swirkernel = get_context_kernel_array(y,x,swircandidates.data)[0]
        nirkernel = get_context_kernel_array(y,x,nircandidates.data)[0] 

        swir = ds[sensor_ignorance[sensor]['2.100-2.300']].data[0][y][x]
        nir = ds[sensor_ignorance[sensor]['0.845-0.885']].data[0][y][x]

        test4.data[0][y][x] = ((swir/nir) > (np.nanmean(swirkernel/nirkernel) + max(3*np.nanstd(swirkernel/nirkernel), 0.8))) 
        test5.data[0][y][x] = (swir > (np.nanmean(swirkernel) + max(3*np.nanstd(swirkernel), 0.08)))

        #print(test4.data[0][y][x],(swir/nir), (np.nanmean(swirkernel/nirkernel) + max(3*np.nanstd(swirkernel/nirkernel), 0.8)))
        #print(test5.data[0][y][x], swir,(np.nanmean(swirkernel) + max(3*np.nanstd(swirkernel), 0.08)) )
        # Write values to new dimension
        #print(index, y, x, get_context_kernel_array(y,x,ds[sensor_ignorance[sensor]['2.100-2.300']].data)[1])
        index = index + 1
    test6 = run_test6(ds)
    hotspots = len(np.where((candidates*(test4*test5*test6)).data == 1))
    return(hotspots, (candidates*(test4*test5*test6)))
#(candidates*(test4*test5*test6)).plot()

In [ ]:
def get_nbr(ds):
    swir = ds[sensor_ignorance[sensor]['2.100-2.300']]
    nir = ds[sensor_ignorance[sensor]['1.560-1.660']]
    return((nir - swir) / (swir + nir))

In [ ]:
get_measurement_list(ds,'ga_s2a_ard_nbar_granule')

In [ ]:
query = {
    'x': xtuple, 
    'y': ytuple,
    'time': (hotspot_solar_day),
    'measurements': measurements,
    'output_crs': 'EPSG:3577',
    'resolution': (-30, 30),
    #'group_by': 'solar_day'
}

# Load available data from Landsat 8 and filter to retain only times
# with at least 99% good data
print(sensors_products[sensor], query)
try:
    ds = load_ard(dc=dc, products=sensors_products[sensor], min_gooddata=0.99, **query)
    #ds = dc.load(products=sensors_products[sensor], **query)
    rgb(ds, bands=[sensor_ignorance[sensor]['2.100-2.300'],sensor_ignorance[sensor]['0.845-0.885'],sensor_ignorance[sensor]['0.450-0.515']], index=0)
except:
    result = '-'


In [ ]:
hotspots, hotspot_array = get_hotspots(ds)
print("This many hotspots have been detected -", hotspots)
hotspot_array.plot()

In [ ]:
nbar1 = nbr(ds)

In [ ]:
nbbr2 = get_nbr(ds)

In [ ]:
((nbbr2[0]-nbar1[0])<-0.09).plot()

In [ ]:
from scipy import ndimage
import xarray

In [ ]:
a = ((nbbr2[0]-nbar1[0])<-0.09)
#ndimage.binary_dilation(a).astype(a.dtype).shape

In [ ]:
type(a)

In [ ]:
b = (a*0).where(ndimage.binary_dilation(a))
#c = (b*0).where(ndimage.binary_dilation(b))

In [ ]:
b.plot()

In [ ]:
c = (a*1).where(ndimage.binary_erosion(dummy))

In [ ]:
c.plot()

https://wiki.landscapetoolbox.org/doku.php/remote_sensing_methods:normalized_burn_ratio

ΔNBR = NBR prefire - NBR postfire

ΔNBR Burn Severity Categories

ΔNBR	Burn Severity
* < -0.25         High post-fire regrowth
* -0.25 to -0.1	  Low post-fire regrowth
* -0.1 to +0.1	  Unburned
* 0.1 to 0.27	  Low-severity burn
* 0.27 to 0.44	  Moderate-low severity burn
* 0.44 to 0.66	  Moderate-high severity burn
* '> 0.66	      High-severity burn

In [ ]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [ ]:
nearest([datetime(2020,1,1), datetime(2020,1,6), datetime(2020,1,12)], datetime(2020,1,5))